# 26. 类设计（Class Design）

类设计强调“职责单一、接口清晰、组合优于继承、依赖抽象”。本节用 ABC 做一个简单接口，并展示依赖注入的好处。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 21-22 节：OOP 与类基础


## 知识点地图

- 1. 职责单一：减少耦合、便于测试
- 2. 组合优于继承：把变化隔离出去
- 3. ABC：用抽象基类表达接口
- 4. 依赖注入：业务类依赖 Storage 接口


## 自检清单（学完打勾）

- [ ] 理解职责单一（一个类做一件事）
- [ ] 能用组合替代不必要的继承
- [ ] 会用 ABC 定义抽象接口（入门）
- [ ] 理解依赖注入：业务类依赖接口而不是具体实现


## 知识点 1：职责单一：减少耦合、便于测试

一个类只负责一类变化；复杂功能拆分成多个协作对象。


## 知识点 2：组合优于继承：把变化隔离出去

通过成员对象组合能力；可替换实现，避免继承层级膨胀。


## 知识点 3：ABC：用抽象基类表达接口

ABC 让接口更明确：实现类必须提供指定方法。


In [ ]:
from abc import ABC, abstractmethod

class Storage(ABC):
    @abstractmethod
    def put(self, key, value):
        raise NotImplementedError

    @abstractmethod
    def get(self, key):
        raise NotImplementedError

class DictStorage(Storage):
    def __init__(self):
        self._d = {}
    def put(self, key, value):
        self._d[key] = value
    def get(self, key):
        return self._d.get(key)

s = DictStorage()
s.put('a', 1)
print(s.get('a'))


## 知识点 4：依赖注入：业务类依赖 Storage 接口

业务类只依赖接口，测试时可替换为内存实现/假实现。


In [ ]:
class Cache:
    def __init__(self, storage):
        self.storage = storage
    def set(self, key, value):
        self.storage.put(key, value)
    def get(self, key):
        return self.storage.get(key)

cache = Cache(DictStorage())
cache.set('x', 42)
print(cache.get('x'))


## 常见坑

- 不要过早抽象：接口太多会增加复杂度
- 继承用于稳定 is-a；变化多的地方优先组合


## 综合小案例：实现 FileStorage（JSON 文件）并接入 Cache

实现 FileStorage：put/get 把数据持久化到 _nb_artifacts/storage.json。


In [ ]:
import json
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)

class FileStorage:
    def __init__(self, path):
        self.path = Path(path)
        if not self.path.exists():
            self.path.write_text('{}', encoding='utf-8')

    def _load(self):
        return json.loads(self.path.read_text(encoding='utf-8'))

    def _save(self, d):
        self.path.write_text(json.dumps(d, ensure_ascii=False, indent=2), encoding='utf-8')

    def put(self, key, value):
        d = self._load()
        d[key] = value
        self._save(d)

    def get(self, key):
        return self._load().get(key)

class Cache:
    def __init__(self, storage):
        self.storage = storage
    def set(self, key, value):
        self.storage.put(key, value)
    def get(self, key):
        return self.storage.get(key)

store = FileStorage(ART / 'storage.json')
cache = Cache(store)
cache.set('a', 1)
print(cache.get('a'))


## 自测题（不写代码也能回答）

- 职责单一为什么重要？
- 依赖注入带来什么好处？
- ABC 与鸭子类型有什么关系？


## 练习题（建议写代码）

- 为 FileStorage 增加 delete(key) 方法。
- 实现一个 MemoryStorage，并与 FileStorage 互换注入到 Cache，验证行为一致。
